In [1]:
import math
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from scipy import stats
import statistics


In [2]:
import tcremb.data_proc as data_proc

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
outputs_path = 'data/data_preped/'

In [5]:
tcr_columns = {'single': ['cdr3aa','v','j','chain'],'paired': {'TRA': ['a_cdr3aa', 'TRAV', 'TRAJ'],'TRB': [ 'b_cdr3aa', 'TRBV', 'TRBJ']}}
tcr_columns = ['cdr3aa','v','j','chain']
clonotype_id_column = 'cloneId'
data_id= 'data_id'
annotation_id = 'annotId'
pairing_id = 'barcode'
annotation_tcr_id_columns_dict = {'TRA': 'cloneId','TRB': 'cloneId','TRA_TRB': {'TRA':'cloneId_TRA', 'TRB':'cloneId_TRB'}}

## Prototypes

### TRA

In [11]:
data_raw_path = 'data/olga_humanTRA_3000.txt'
data_output_path = outputs_path +'olga_humanTRA.txt'

In [12]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [13]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(2951, 4)

In [14]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

### TRB

In [15]:
data_raw_path = 'data/olga_humanTRB_3000.txt'
data_output_path = outputs_path +'olga_humanTRB.txt'

In [16]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [17]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(3000, 4)

In [18]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

## VDJdb

In [6]:
label = 'antigen.epitope'
label_s = 'antigen.species'

In [7]:
data_raw_path = 'data/vdjdb.slim.txt'
data_tra_output_path = outputs_path +'VDJdb_data_TRA.csv'
data_trb_output_path = outputs_path +'VDJdb_data_TRB.csv'
data_output_path = outputs_path +'VDJdb_data.csv'

In [8]:
data = pd.read_csv(data_raw_path,sep='\t')
data = data.rename({'cdr3':'cdr3aa','gene':'chain','v.segm':'v','j.segm':'j'},axis=1)




### Assign data id

In [9]:
data = data_proc.annot_id(data, data_id)

### data proc

In [10]:
data_preped = data_proc.remove_asterisk(data, tcr_columns)
data_preped = data_proc.remove_backslash(data_preped, tcr_columns)
data_preped = data_proc.filter_clones_data(data_preped, tcr_columns)
data_preped = data_preped[-data_preped['reference.id'].str.startswith('https://www.10xgenomics',na=False)].reset_index(drop=True)
data_preped = data_preped[data_preped['species']=='HomoSapiens']
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(75280, 17)
(74240, 17)


(36193, 17)

### frequent labels

In [13]:
data_preped = data_proc.freq_label(label, data_id, data_preped, tr = 1000)
data_preped = data_proc.freq_label_list(label, data_id, data_preped, [10,20,30,50,100,500,1000])

In [19]:
data_preped = data_proc.freq_label(label_s, data_id, data_preped, tr = 100)
data_preped = data_proc.freq_label_list(label_s, data_id, data_preped, [10,20,30,50,100,500,1000])

### save

In [26]:
data_preped.to_csv(data_output_path, sep='\t',index=False)

In [ ]:
data_preped_tra = data_preped[data_preped['chain']=='TRA'].reset_index(drop=True)

In [ ]:
data_preped_trb = data_preped[data_preped['chain']=='TRB'].reset_index(drop=True)

## 10x donor 1